# Data schoonmaken

Voor dit project hebben we vier verschillende datasets opgeschoond en samengevoegd. Het eerste en meest uitdagende aspect van deze taak was het harmoniseren van de landnamen. Elk van de datasets gebruikte verschillende namen of codes voor landen, wat directe vergelijking onmogelijk maakte. Om dit op te lossen, hebben we een uniforme lijst van landennamen opgesteld en vervolgens een mapping uitgevoerd om de inconsistenties tussen de datasets te corrigeren. Dit zorgde ervoor dat elk land in alle datasets dezelfde naam kreeg, waardoor een nauwkeurige samensmelting mogelijk werd.
Vervolgens hebben we alle datasets geïnspecteerd om overbodige kolommen te identificeren en te verwijderen. Deze kolommen bevatten gegevens die niet relevant waren voor onze analyse of duplicaten van informatie die al in andere kolommen aanwezig was. Door deze kolommen te verwijderen, hebben we een strakkere, beter beheersbare dataset gekregen.
Na het opschonen van de individuele datasets, hebben we deze samengevoegd op basis van de uniforme landnamen en jaar. De datasets overlappen alleen voor een bepaalde periode van tijd. Dit is de periode van 2001 tot 2012. We hebben de datasets samengevoegd op deze jaren en overbodige jaren verwijderd. Hierdoor ontstond een enkele, geïntegreerde dataset die alle benodigde informatie bevatte zonder overbodige informatie. Deze uiteindelijke dataset bevatte alleen de relevante kolommen die we nodig hadden voor onze analyse, wat de efficiëntie en nauwkeurigheid van ons werk aanzienlijk verbeterde.

Hieronder staan de datasets die wij gebruikt hebben:

Dataset over kankertype en doden: https://www.kaggle.com/datasets/antimoni/cancer-deaths-by-country-and-type-1990-2016 
The Cancer Deaths by Country and Type dataset bevat data van 18 verschillende soorten kanker (prostaatkanker, leverkanker, TBL-kanker etc.) en het dodental per soort kanker per jaar en per land. De data is van 1990-2016. De data is verzameld van de WHO (World Health Organisation).

Deze data gaan we vervolgens naast de dataset over roken leggen: 
https://www.kaggle.com/datasets/mexwell/us-smoking-trend
De Global Smoking Trend dataset bevat data over het aantal rokers per land per jaar. De dataset is van 1980-2012. 

En daarnaast vergelijken met de dataset van uitstoot:
https://www.kaggle.com/datasets/thedevastator/global-fossil-co2-emissions-by-country-2002-2022
De Emissions by Country dataset bevat mondiale data van fossiele brandstof CO2-uitstoot per land per jaar. De dataset bevat totale CO2-uitstoot per jaar per land maar ook uitstoot per aandeel gas, olie, kolen of uitstoot per inwoner. De dataset is van 1750-2021.

Om data om te zetten naar per capita data, hebben we een dataset gevonden met populatie per land per jaar:
https://github.com/datasets/population/blob/main/data/population.csv 


Continuous / Ratio variabelen: Rokers, TBL-Kanker, Emissies
Discrete / Nominale variabelen: Land
Discrete / Interval variabelen: Jaar

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go 
from plotly.subplots import make_subplots

# Data inladen

Eerst worden de juiste datasets ingeladen, dit zijn de vier boven benoemde datasets.

In [2]:
cancer_df = pd.read_csv("csv/CancerDeaths.csv")
emissions_df = pd.read_csv("csv/Emissions.csv")
population_df = pd.read_csv("csv/Population.csv")
smoking_df = pd.read_csv("csv/smoking.csv")

# Data opschonen

Hieronder worden de datasets 'schoongemaakt' en opgeslagen in het csv mapje. 

## Vergelijking uitstoot en TBL Kanker

Onderstaande dataset wordt gecleaned op de datasets van TBL-kanker en Totale emissies. Het wordt vervolgens gecombineerd tot de file 'emissions_cancer_merged'. Deze zal gebruikt worden voor de subplots. De variabelen die gemerged worden zijn 'Year', 'Tracheal, bronchus, and lung cancer ', en 'Total' (totale emissies).


In [3]:
# df voor uitstoot en TBL

# Filter the DataFrame for the desired country and year range
cancer_country = 'World'  # Replace with the desired country code
emissions_country = 'Global'  # Replace with the desired country code

cancer_data = cancer_df[(cancer_df['Country'] == cancer_country) & (cancer_df['Year'] >= 2001)]
emissions_data = emissions_df[(emissions_df['Country'] == emissions_country) & (emissions_df['Year'] >= 2001)]
cancer_data.head()
# Extract the Year and Lung cancer columns
year_lung_cancer = cancer_data[['Year', 'Tracheal, bronchus, and lung cancer ']]
year_lung_cancer = year_lung_cancer[year_lung_cancer['Year'] <= 2012]

year_total_emissions = emissions_data[['Year', 'Total']]
year_total_emissions = year_total_emissions[year_total_emissions['Year'] <= 2012]

year_total_output = pd.merge(year_lung_cancer, year_total_emissions)

# Sla de opgeschoonde data op

output_file_total = 'csv/df/emissions_cancer_merged.csv'

year_total_output.to_csv(output_file_total, index=False)

year_total_output.head()



,Year,"Tracheal, bronchus, and lung cancer",Total
0,2001,1306846.723,25668.048326
1,2002,1340682.674,26281.037468
2,2003,1365249.848,27651.596453
3,2004,1393709.471,28636.695037
4,2005,1432109.236,29614.602256


## Wereldkaartplots en USA/India vergelijkingsplot

De tweede dataset zal gebruikt worden voor de chloroplets en de vergelijking tussen USA en India. De overeenkomende variabelen waren 'year' en 'country', en uit de emissions-, cancer-, en smokingdataset worden respectievelijk 'Data.Smokers.Total', 'Tracheal, bronchus, and lung cancer' en 'Total' gehaald. Deze worden vervolgens per variabele gedeeld door de variabele 'PopTotal' van de populatie dataset. Dit hebben we gedaan zodat we de per capita values in de dataset konden zetten. Vervolgens hebben we het gemerged in de 'combined' dataset en opgeslagen als nieuwe csv.

In [4]:
# Data cleaning voor wereldkaartplots en USA/India vergelijkingsplot

# Load the datasets
df_lung = cancer_df
df_emissions = emissions_df
df_population = population_df
df_smoking = smoking_df

# Strip any leading/trailing whitespace from column names
df_lung.columns = df_lung.columns.str.strip()
df_emissions.columns = df_emissions.columns.str.strip()
df_population.columns = df_population.columns.str.strip()
df_smoking.columns = df_smoking.columns.str.strip()

# Standardize country names using a mapping dictionary
country_name_mapping = {
    'Russian Federation': 'Russia',
    'United States of America': 'USA',
    'United States': 'USA'
    # Add other mappings if necessary
}

# Apply the mapping to the emissions dataset
df_emissions['Country'] = df_emissions['Country'].replace(country_name_mapping)

# Apply the mapping to the lung cancer dataset
df_lung['Country'] = df_lung['Country'].replace(country_name_mapping)

df_smoking['Country'] = df_smoking['Country'].replace(country_name_mapping)

df_population['Location'] = df_population['Location'].replace(country_name_mapping)

# Filter data to include only years 2001-2012
df_lung_filtered = df_lung[(df_lung['Year'] >= 2000) & (df_lung['Year'] <= 2012)]
df_emissions_filtered = df_emissions[(df_emissions['Year'] >= 2000) & (df_emissions['Year'] <= 2012)]
df_smoking_filtered = df_smoking[(df_smoking['Year'] >= 2000) & (df_smoking['Year'] <= 2012)]

# Merge the lung cancer data with population data
df_lung_merged = pd.merge(df_lung_filtered, df_population, left_on=['Country', 'Year'], right_on=['Location', 'Time'])
df_lung_merged['Lung Cancer Per Capita'] = df_lung_merged['Tracheal, bronchus, and lung cancer'] / df_lung_merged['PopTotal']

# Merge the emissions data with population data
df_emissions_merged = pd.merge(df_emissions_filtered, df_population, left_on=['Country', 'Year'], right_on=['Location', 'Time'])
df_emissions_merged['Emissions Per Capita'] = df_emissions_merged['Total'] / df_emissions_merged['PopTotal']

# Merge the smoking data with population data
df_smoking_merged = pd.merge(df_smoking_filtered, df_population, left_on=['Country', 'Year'], right_on=['Location', 'Time'])
df_smoking_merged['Smokers Per Capita'] = df_smoking_merged['Data.Smokers.Total'] / df_smoking_merged['PopTotal']

# Aggregate data by country, year, and code
df_lung_per_capita = df_lung_merged.groupby(['Country', 'Year'])['Lung Cancer Per Capita'].mean().reset_index()
df_emissions_per_capita = df_emissions_merged.groupby(['Country', 'Year'])['Emissions Per Capita'].mean().reset_index()
df_smoking_per_capita = df_smoking_merged.groupby(['Country', 'Year'])['Smokers Per Capita'].mean().reset_index()

# Rename the columns for better readability
df_lung_per_capita.columns = ['Country', 'Year', 'Lung Cancer Per Capita']
df_emissions_per_capita.columns = ['Country', 'Year', 'Emissions Per Capita']
df_smoking_per_capita.columns = ['Country', 'Year', 'Smokers Per Capita']

# Merge the two datasets on country code and year
df_combined = pd.merge(df_lung_per_capita, df_emissions_per_capita, on=['Country', 'Year'])
df_combined = pd.merge(df_combined, df_smoking_per_capita, on=['Country', 'Year'])

output_file_combined = 'csv/df/combined.csv'
df_combined.to_csv(output_file_combined, index=False)

df_combined.head()

,Country,Year,Lung Cancer Per Capita,Emissions Per Capita,Smokers Per Capita
0,Afghanistan,2000,0.000071,5.358077e-08,0.062883
1,Afghanistan,2001,0.000074,5.430027e-08,0.064753
2,Afghanistan,2002,0.000072,6.385613e-08,0.063324
3,Afghanistan,2003,0.000070,6.887141e-08,0.061384
4,Afghanistan,2004,0.000071,5.252911e-08,0.061540


## Correlatieplot Emissie en TBL-kanker

Onderstaand wordt een dataset gecreëerd met de variabelen 'LungCancerPerCapita', 'EmissionsPerCapita' en 'Year', deze zal worden gebruikt voor het maken van het correlatieplot.

In [5]:
# df's voor correlatieplot emissie en tbl

# Filter data to include only years 2000-2012
df_lung_filtered = df_lung[(df_lung['Year'] >= 2000) & (df_lung['Year'] <= 2012)]
df_emissions_filtered = df_emissions[(df_emissions['Year'] >= 2000) & (df_emissions['Year'] <= 2012)]
df_population_filtered = df_population[(df_population['Time'] >= 2000) & (df_population['Time'] <= 2012)]

# Extract relevant columns
df_lung_relevant = df_lung_filtered[['Country', 'Year', 'Tracheal, bronchus, and lung cancer']]
df_emissions_relevant = df_emissions_filtered[['Country', 'Year', 'Total']]
df_population_relevant = df_population_filtered[['Location', 'Time', 'PopTotal']]

# Merge datasets on Country and Year
df_merged = pd.merge(df_lung_relevant, df_emissions_relevant, left_on=['Country', 'Year'], right_on=['Country', 'Year'])
df_merged = pd.merge(df_merged, df_population_relevant, left_on=['Country', 'Year'], right_on=['Location', 'Time'])

# Calculate per capita values
df_merged['LungCancerPerCapita'] = df_merged['Tracheal, bronchus, and lung cancer'] / df_merged['PopTotal']
df_merged['EmissionsPerCapita'] = df_merged['Total'] / df_merged['PopTotal']

# Select only the numeric columns
df_numeric = df_merged[['Year', 'LungCancerPerCapita', 'EmissionsPerCapita']]

# Group by Year and average the per capita values
df_grouped = df_numeric.groupby('Year').mean().reset_index()

output_file_grouped = 'csv/df/grouped_df.csv'
df_grouped.to_csv(output_file_grouped, index=False)

df_grouped.head()

,Year,LungCancerPerCapita,EmissionsPerCapita
0,2000,0.000191,0.000005
1,2001,0.000192,0.000005
2,2002,0.000193,0.000005
3,2003,0.000195,0.000005
4,2004,0.000197,0.000005


## Correlatieplot roken en longkanker

Hieronder wordt de dataset gecleaned voor de variabelen 'Year', 'LungCancerPerCapita', 'SmokersPerCapita' en 'CigarettesPerCapita'. Deze zullen gebruikt worden om de correlatieplot voor deze variabelen te creëeren.

In [6]:
# correlatie plot roken en longkanker

# Extract relevant columns
df_smoking_relevant = df_smoking_filtered[['Country', 'Year', 'Data.Smokers.Total', 'Data.Daily cigarettes']]

# Merge datasets on Country and Year
df_merged = pd.merge(df_lung_relevant, df_smoking_relevant, left_on=['Country', 'Year'], right_on=['Country', 'Year'])
df_merged = pd.merge(df_merged, df_population_relevant, left_on=['Country', 'Year'], right_on=['Location', 'Time'])

# Calculate per capita values
df_merged['LungCancerPerCapita'] = df_merged['Tracheal, bronchus, and lung cancer'] / df_merged['PopTotal']
df_merged['SmokersPerCapita'] = df_merged['Data.Smokers.Total'] / df_merged['PopTotal']
df_merged = df_merged.rename(columns={'Data.Daily cigarettes': 'CigarettesPerCapita'})

# Select only the numeric columns
df_numeric = df_merged[['Year', 'LungCancerPerCapita', 'SmokersPerCapita', 'CigarettesPerCapita']]

# Group by Year and average the per capita values
df_grouped = df_numeric.groupby('Year').mean().reset_index()

output_file_grouped = 'csv/df/grouped_smokers_df.csv'
df_grouped.to_csv(output_file_grouped, index=False)

df_grouped.head()

,Year,LungCancerPerCapita,SmokersPerCapita,CigarettesPerCapita
0,2000,0.000187,0.131518,18.491975
1,2001,0.000189,0.130765,18.483951
2,2002,0.000190,0.129956,18.470370
3,2003,0.000192,0.129062,18.472222
4,2004,0.000193,0.128358,18.438272
